#### Code to extract SEC filings 
- This code retrieves recent 10-K, 10-Q, and 8-K filings from the SEC's EDGAR database for a given list of companies y their CIK numbers.
- The code sends HTTP requests to the SEC API, processes the responses, and filters filings for the specified forms (10-K, 10-Q, 8-K). These forms were relevant to the scope of our RAG project
- The relevant details, including the filing date, form type, accession number (id for document), and document URL, are extracted and compiled into a pandas DataFrame.


In [1]:
import requests
import pandas as pd

def list_filings(cik_list):
    """
    List all 10-K, 10-Q, and 8-K filings along with their relevant fields for a given list of CIK numbers
    from the SEC's EDGAR database and return it as a structured table.
    CIK numbers represent a company

    Args:
        cik_list (list): List of CIK numbers (e.g., ["0000732717", "0000320193", "66740"]).

    Returns:
        pd.DataFrame: DataFrame containing all filings and their details for each CIK in the list.
    """
    base_url = "https://data.sec.gov/submissions/CIK{}.json"
    headers = {'User-Agent': 'MyAppName/1.0 (myemail@example.com)'}  # optional: replace with your app name and email

    all_filings_data = []  # List to hold data for all CIKs

    for cik in cik_list:
        # Remove leading zeros and ensure CIK is formatted as a string
        cik_str = str(cik).lstrip('0')

        # Get company filings data from SEC EDGAR database
        filings_url = base_url.format(str(cik).zfill(10))  # Fill the CIK to 10 digits for the request
        response = requests.get(filings_url, headers=headers)
        if response.status_code != 200:
            print(f"Error retrieving filings for CIK {cik_str}.")
            continue

        # Load the company's submissions
        submissions = response.json()
        filings = submissions.get("filings", {}).get("recent", {})

        # Check if required keys are present and lists are not empty
        required_keys = ["form", "accessionNumber", "filingDate", "primaryDocument", "primaryDocDescription", "size"]
        if not filings or not all(key in filings for key in required_keys):
            print(f"Filings data structure is missing or malformed for CIK: {cik_str}.")
            continue

        # Filter forms for '10-K', '10-Q', and '8-K'
        form_list = ['10-K', '10-Q', '8-K']
        for index, form in enumerate(filings.get("form", [])):
            if form in form_list:
                accession_number = filings.get("accessionNumber", [None])[index]
                
                # Construct the document URL based on the EDGAR filing format
                document_url = f"https://www.sec.gov/Archives/edgar/data/{cik_str}/{accession_number.replace('-', '')}/{filings.get('primaryDocument', [None])[index]}"
                
                all_filings_data.append({
                    "Ticker": submissions.get("tickers", [None])[0],  # Taking the first ticker if multiple are present
                    "Provided CIK": cik_str,  # Adding the cleaned CIK to each row
                    "CIK": cik_str,  # Cleaned CIK without leading zeros
                    "SIC": submissions.get("sic", ""),
                    "SIC Description": submissions.get("sicDescription", ""),
                    "Accession Number": accession_number,
                    "Filing Date": filings.get("filingDate", [None])[index],
                    "Form": form,
                    "Size": filings.get("size", [None])[index],
                    "Primary Document": filings.get("primaryDocument", [None])[index],
                    "Primary Document Description": filings.get("primaryDocDescription", [None])[index],
                    "Document URL": document_url  # URL to access the HTML document
                })

    # Convert all collected filings data into a single DataFrame
    all_filings_df = pd.DataFrame(all_filings_data)
    if all_filings_df.empty:
        print(f"No filings found for the provided CIKs.")
        return None

    print(f"Total SEC filings found: {len(all_filings_df)}")
    return all_filings_df

# Example CIKs for Magnificent Seven
magnificient7 = ["320193", "789019", "1018724", "1045810", "1652044", "1326801", "1318605"]
# 320193 = Apple, 789019 = Microsoft, 1018724 = Amazon, 1045810 = Nvidia, 1326801 = Meta, 1318605 = Tesla, 1652044 = Google

result_df = list_filings(magnificient7)
result_df.head(40)


C:\Users\chelero\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\chelero\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\chelero\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Total SEC filings found: 708


,Ticker,Provided CIK,CIK,SIC,SIC Description,Accession Number,Filing Date,Form,Size,Primary Document,Primary Document Description,Document URL
0,AAPL,320193,320193,3571,Electronic Computers,0000320193-24-000123,2024-11-01,10-K,9759333,aapl-20240928.htm,10-K,https://www.sec.gov/Archives/edgar/data/320193...
1,AAPL,320193,320193,3571,Electronic Computers,0000320193-24-000120,2024-10-31,8-K,469716,aapl-20241031.htm,8-K,https://www.sec.gov/Archives/edgar/data/320193...
2,AAPL,320193,320193,3571,Electronic Computers,0001140361-24-040659,2024-09-10,8-K,1248169,ef20035718_8k.htm,FORM 8-K,https://www.sec.gov/Archives/edgar/data/320193...
3,AAPL,320193,320193,3571,Electronic Computers,0001140361-24-038601,2024-08-26,8-K,1242096,ny20033611x3_8k.htm,FORM 8-K,https://www.sec.gov/Archives/edgar/data/320193...
4,AAPL,320193,320193,3571,Electronic Computers,0001140361-24-038403,2024-08-23,8-K,1131218,ny20033611x2_8k.htm,8-K,https://www.sec.gov/Archives/edgar/data/320193...
5,AAPL,320193,320193,3571,Electronic Computers,0000320193-24-000081,2024-08-02,10-Q,5372771,aapl-20240629.htm,10-Q,https://www.sec.gov/Archives/edgar/data/320193...
6,AAPL,320193,320193,3571,Electronic Computers,0000320193-24-000080,2024-08-01,8-K,437770,aapl-20240801.htm,8-K,https://www.sec.gov/Archives/edgar/data/320193...
7,AAPL,320193,320193,3571,Electronic Computers,0001140361-24-024352,2024-05-03,8-K,1131483,ef20028273_8k.htm,8-K,https://www.sec.gov/Archives/edgar/data/320193...
8,AAPL,320193,320193,3571,Electronic Computers,0000320193-24-000069,2024-05-03,10-Q,5284139,aapl-20240330.htm,10-Q,https://www.sec.gov/Archives/edgar/data/320193...
9,AAPL,320193,320193,3571,Electronic Computers,0000320193-24-000067,2024-05-02,8-K,436360,aapl-20240502.htm,8-K,https://www.sec.gov/Archives/edgar/data/320193...


In [4]:
##saving the list of documents for metadata purpose as needed for RAG retrieval
import pandas as pd

#File path in G drive , change accordingly 
file_path = r"G:\.shortcut-targets-by-id\1Kl4a21psV3k0KFTuAD9KyqSJlzx89eNz\Capstone\sec_filings\sec_filings.xlsx"

# Save as Excel
result_df.to_excel(file_path, index=False, engine='openpyxl')
print(f"File saved successfully at {file_path}.")

File saved successfully at G:\.shortcut-targets-by-id\1Kl4a21psV3k0KFTuAD9KyqSJlzx89eNz\Capstone\sec_filings\sec_filings.xlsx.


#### code below is by company - we targeted only 7 companies
Pulling SEC filings company by company will not be blocked by SEC API

In [ ]:
import pandas as pd
import os
import requests

# path where files will be saved
output_dir = r"G:\.shortcut-targets-by-id\1Kl4a21psV3k0KFTuAD9KyqSJlzx89eNz\Capstone\sec_filings"

# output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Ensure 'Filing Date' is in datetime format
result_df['Filing Date'] = pd.to_datetime(result_df['Filing Date'], errors='coerce')

# Drop rows where 'Filing Date' could not be converted to datetime
result_df = result_df.dropna(subset=['Filing Date'])

# Filter the DataFrame to include only filings after October 1, 2022 - 2 years is more than enough relevant information
filtered_df = result_df[result_df['Filing Date'] > pd.to_datetime('2022-10-01')]

# Iterate through unique companies (tickers) in the filtered DataFrame
for ticker in filtered_df['Ticker'].unique():
    # Filter data for the current company
    company_df = filtered_df[filtered_df['Ticker'] == ticker]
    
    # Create a directory for the company within the base output directory
    company_dir = os.path.join(base_output_dir, ticker)
    os.makedirs(company_dir, exist_ok=True)
    
    # Extract necessary columns for downloading and naming files
    document_urls = company_df['Document URL']
    ciks = company_df['CIK']
    filing_dates = company_df['Filing Date']
    forms = company_df['Form']
    
    # Function to download a file from a URL with a custom name
    def download_file(url, cik, filing_date, form, directory):
        # Create a custom filename using Ticker, CIK, Filing Date, and Form
        local_filename = os.path.join(
            directory, f"{ticker}_{cik}_{filing_date.strftime('%Y-%m-%d')}_{form}.html"
        )
        headers = {'User-Agent': 'MyAppName/1.0 (myemail@example.com)'}
        
        try:
            # Send HTTP GET request with custom User-Agent
            with requests.get(url, stream=True, headers=headers) as response:
                response.raise_for_status()
                # Write to a local file
                with open(local_filename, 'wb') as file:
                    for chunk in response.iter_content(chunk_size=8192):
                        file.write(chunk)
            print(f"Downloaded: {local_filename}")
        except Exception as e:
            print(f"Failed to download {url}: {e}")
    
    # Download all files for the current company
    for index, url in enumerate(document_urls):
        download_file(
            url,
            ciks.iloc[index],
            filing_dates.iloc[index],
            forms.iloc[index],
            company_dir
        )
